### Your name:

<pre> Peihang Xu</pre>

### Collaborators:

<pre> N/A</pre>


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Open the housing data


In [2]:
import os
import tarfile
from six.moves import urllib
import pandas as pd


DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
print("DATA SHAPE:", housing.shape)
housing.head()

DATA SHAPE: (20640, 10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Build full pipeline for the data analysis following the example of the notebook.
 Hint: the main part requested to change is the algorithm used (KNN regression)


#### Considerations for building pipeline:

- Make your notebook as compact as possible. 
- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform KNN regression using a variety of values for n_neighbors (K) between 1 and 10 and both "uniform" and "distance" weights via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

#### Stratified split the train and test set

In [3]:
# Divide by 1.5 to limit the number of income categories 
# adding a new attribute here called "income_cat"
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0,inplace = True)

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [5]:
#sanity check
from sklearn.model_selection import train_test_split
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
}).sort_index()
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100
compare_props

,Overall,Stratified,Strat. %error
1.0,0.039826,0.039729,-0.243309
2.0,0.318847,0.318798,-0.015195
3.0,0.350581,0.350533,-0.013820
4.0,0.176308,0.176357,0.027480
5.0,0.114438,0.114583,0.127011


In [6]:
# drop the "income_cat" attribute after splitting
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

strat_train_set.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,286600.0,<1H OCEAN
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,340600.0,<1H OCEAN
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,196900.0,NEAR OCEAN
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,46300.0,INLAND
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,254500.0,<1H OCEAN


#### Create a full transformation pipeline to impute missing components, convert all categorical data to one-hot vectors, and normalize all non-categorical data

In [7]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

In [12]:
# Firstly impute missing values and do min-max scaling
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

housing_num = housing.drop('ocean_proximity', axis=1) # Define a dataframe with only numerical attributes
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"] # Lists of numerical/categorical attribute names

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), #impute
        ('std_scaler', MinMaxScaler(feature_range=(0, 1))), # then normalize
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num) #transform housing_num daata frame to an np array

In [14]:
# Here define a full pipeline transformer to transform the whole training dataset all at once
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared.shape

(16512, 13)

#### Implement KNN regression

In [15]:
from sklearn.neighbors import KNeighborsRegressor

KNN_reg = KNeighborsRegressor()
KNN_reg.fit(housing_prepared, housing_labels)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [16]:
# Cross-validation for 10-fold

from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())


KNN_scores = cross_val_score(KNN_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
KNN_rmse_scores = np.sqrt(-KNN_scores)
display_scores(KNN_rmse_scores)



Scores: [60650.94168635 62650.20957684 63775.61883886 63071.65536651
 62127.10348079 64943.97508527 60860.88448428 63456.28743377
 63959.71438229 64183.52086099]
Mean: 62967.99111959522
Standard deviation: 1337.2986163439268


In [17]:
# or use this to display the cross validation results
scores = cross_val_score(KNN_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

count       10.000000
mean     62967.991120
std       1409.636513
min      60650.941686
25%      62257.880005
50%      63263.971400
75%      63913.690496
max      64943.975085
dtype: float64

#### Grid search

In [18]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    # try 4 (2×2) combinations of hyperparameters
    {'n_neighbors': [1, 10], 'weights': ['uniform','distance']}
  ]

grid_search = GridSearchCV(KNN_reg, param_grid, cv=10, #totally 40 rounds of training since 2x2x10
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)
grid_search.best_params_

{'n_neighbors': 10, 'weights': 'distance'}

In [19]:
cvres = grid_search.cv_results_
cvres["mean_test_score"]
cvres["params"]
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

76075.60050887972 {'n_neighbors': 1, 'weights': 'uniform'}
76075.60050887972 {'n_neighbors': 1, 'weights': 'distance'}
62355.70276799552 {'n_neighbors': 10, 'weights': 'uniform'}
61560.95930357948 {'n_neighbors': 10, 'weights': 'distance'}


#### Use with test set

In [20]:
from sklearn.metrics import mean_squared_error

final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test) # no fit here!
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse
final_predictions

array([456014.43182275, 279271.4193661 , 269806.65401908, ...,
       296076.79043123, 160226.91641625, 104495.70479672])

In [21]:
grid_search.best_params_

{'n_neighbors': 10, 'weights': 'distance'}

In [22]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
mean = squared_errors.mean()
m = len(squared_errors)

np.sqrt(stats.t.interval(confidence, m - 1,
                         loc=np.mean(squared_errors),
                         scale=stats.sem(squared_errors)))

array([58209.0555843 , 62683.38636523])

### A full pipeline with both preparation and prediction 

In [23]:

prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('KNN_reg', KNeighborsRegressor(**grid_search.best_params_))
])
prepare_select_and_predict_pipeline.fit(housing,housing_labels)

Pipeline(memory=None,
         steps=[('preparation',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                              

In [24]:
print("Predictions:\t", prepare_select_and_predict_pipeline.predict(X_test))

Predictions:	 [456014.43182275 279271.4193661  269806.65401908 ... 296076.79043123
 160226.91641625 104495.70479672]


### Conclusions
For what values of n_neighbors and weight does KNeighborsRegressor perform the best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?

<pre> 
I found the best hyperparameter values to be n_neighbors = 10 and weights = 'distance'.

The final mean score of the two algorithms gives 69052.46136345083 for linear regressor and 61560.95930357948    for KNeighborsRegressor after grid search.

By comparing them, I think KNeighborsRegressor performs slightly better than linear regressor due to a lower      value of score.

The difference here can be caused by a few reasons: 
   1. in my homework calculation, I do not include some added attributes such as rooms_per_household and
      population_per_household
   2. for linear regressor, in the class we did not try to fine-tune the hyperparameters using grid search or 
      random search
   3. The feature scaling method used here is noramalization (min-max scaler) instead of standarzation, which         
      means it could be more affected by outliers 
   4. the property of the data itself may just determines that it would work better with KNeighborRegressor 
   

   
</pre>

### Read appending B

- Reflect on your last data project, read appendix B. Then, write down a few of the checklist items that your last data project could have used. If you have not yet done a data project, then write down a few of the items that you found most interesting.


<pre>

I am still a physics Ph.D. candidate and very new to this field. So I have never really done a data project. In my physics study, I take and analyze experimental data, and usually just use matlab to do a fit to the theory     model.
However in the data science field, all data are more realistic and fitting results can be applied to everyday   life. 
So in this application, trying to clean the data, find a good machine learning algorithm, and tune the model are critical. They play an important role for accuracy of the prediction. The predictions can then be applied to    important applications such as decisions for investments. 


 

<pre>

### Submit your notebook

Submit your solution to Quercus
Make sure you rename your notebook to    
W2_UTORid.ipynb    
Example W2_adfasd01.ipynb
